In [15]:
print("ok")

ok


In [16]:
from langchain.prompts import PromptTemplate
from langchain.chains import retrieval_qa
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import pinecone as pcn
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import ctransformers
import pinecone


In [17]:
PINECONE_API_KEY = "df3c7b7f-791e-43c0-bfbb-05d1423a1cfd"
PINECONE_API_ENV = "us-east-1"


In [18]:
#extract data from the pdf 

def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents

In [19]:
extracted_data = load_pdf("data/")

### Now we have to convert our corpus to chunks to process into out vector database 

In [20]:
def text_splitter(extracted_data):
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    text_chunks = splitter.split_documents(extracted_data)

    return text_chunks

In [21]:
text_chunks = text_splitter(extracted_data)
print("number of chunks : ", len(text_chunks))

number of chunks :  7151


## now we have to conver our text chunks into vector

In [22]:
#donloading embedding model 

def download_hugging_face_embedding():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    return embeddings

In [23]:
embedding = download_hugging_face_embedding()

In [24]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [25]:
type(text_chunks[0].page_content)

str

In [26]:
test = embedding.embed_query(text_chunks[0].page_content)
len(test)


384

In [27]:
type(test)

list

In [28]:
vectors = []
i=0
for chunk in text_chunks:
    {"id": "vec1", "values": [1.0, 1.5]},
    embd = embedding.embed_query(chunk.page_content)
    my_dict = {"id": "vec"+str(i), "values": embd}
    i = i+1
    vectors.append(my_dict)





In [29]:
vectors[0]

{'id': 'vec0',
 'values': [0.0017461127135902643,
  -0.03350285068154335,
  -0.03290392458438873,
  0.007168037351220846,
  -0.014603203162550926,
  0.010261881165206432,
  -0.01151531096547842,
  0.22930212318897247,
  -0.02323242276906967,
  0.0041204276494681835,
  -0.03656088188290596,
  0.08592100441455841,
  0.012972156517207623,
  0.05221784859895706,
  -0.10232611000537872,
  -0.0031389212235808372,
  -0.012686879374086857,
  0.00047185696894302964,
  -0.028485868126153946,
  -0.05025916546583176,
  0.011551006697118282,
  0.07780643552541733,
  0.09282826632261276,
  -0.013797217048704624,
  -0.016935087740421295,
  -0.02595587447285652,
  -0.04956516623497009,
  -0.04613134637475014,
  0.0072905332781374454,
  -0.013553272932767868,
  0.03843948245048523,
  0.06280473619699478,
  0.018353857100009918,
  0.008242896758019924,
  0.0017155351815745234,
  -0.039861805737018585,
  -0.011638658121228218,
  0.0164461899548769,
  0.02559553273022175,
  0.09104610979557037,
  0.029672

In [30]:
PINECONE_API_KEY = 'bba9f306-8d5e-40d6-bba2-41e517d1f8af'

from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

In [31]:
# pc.create_index(
#     name="medbot",
#     dimension=384, # Replace with your model dimensions
#     metric="cosine", # Replace with your model metric
#     spec=ServerlessSpec(
#         cloud="aws",
#         region="us-east-1"
#     ) 
# )

In [32]:
len(vectors)

7151

In [36]:
index = pc.Index("medbot")
# i = 0
# for vec in vectors:
#     index.upsert(
#         vectors=[vec]
#     )
#     print(i)
#     i=i+1

In [37]:
print(index.describe_index_stats())

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 7151}},
 'total_vector_count': 7151}


## running a similarity search

In [42]:
ques = "what is the meaning of acne?"
ques_vec = embedding.embed_query(ques)

query_results1 = index.query(
    vector=ques_vec,
    top_k=3,
    include_values=True
)

query_results1

{'matches': [{'id': 'vec320',
              'score': 0.684680164,
              'values': [-0.0579594374,
                         0.01088179,
                         -0.0548172966,
                         0.0652917698,
                         0.0296164826,
                         0.0147812953,
                         0.0392949954,
                         0.0740290508,
                         -0.0198292974,
                         -0.0734772384,
                         0.039435219,
                         -0.0507786646,
                         0.0531869605,
                         -0.0301832911,
                         -0.0404771194,
                         -0.0374963209,
                         -0.0515484,
                         -0.0282289907,
                         -0.030486472,
                         -0.0276594125,
                         -0.0202243701,
                         0.0755395293,
                         -0.0309191756,
                         -0.11

In [43]:
import time
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index,
    embedding=embedding
)

time.sleep(1)


PineconeConfigurationError: You haven't specified an Api-Key.

In [ ]:
from langchain_pinecone import PineconeVectorStore


NameError: name 'langchain_pinecone' is not defined